In [ ]:

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 1. Import Required Modules

import os
import glob
import keras
import random
import numpy as np
import tensorflow as tf
from keras.layers import *
import keras.backend as k
from keras.models import *
from keras.optimizers import *
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread, imshow, imsave, imread_collection
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping

In [ ]:
# 3. Initialize Images and Mask Size

IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 512, 512, 1

In [ ]:
import glob
import cv2

Train_Input = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Image_512x512/*.png"))]
Train_Mask = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Label_512x512/*.png"))]

Train_Input = np.array(Train_Input)
Train_Mask = np.array(Train_Mask)

Train_Mask = cv2.normalize(Train_Mask, None, alpha=1,beta=0, norm_type=cv2.NORM_MINMAX)

In [ ]:
Train_Input = Train_Input[:2500]
Train_Mask = Train_Mask[:2500]

In [ ]:
Train_Mask.shape

In [ ]:
imshow(Train_Input[1300])

In [ ]:
imshow(Train_Mask[1300], cmap='Greys_r')

In [ ]:
Train_Mask.shape

In [ ]:
print('Training Input')
imshow(Train_Input[45])
plt.show()

print('Training Mask')
imshow(np.squeeze(Train_Mask[45]), cmap='Greys_r')
plt.show()


In [ ]:
print('Training Input')
imshow(Train_Input[45])
plt.show()

In [ ]:
def dice_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)

    return 1 - numerator / denominator

def jacard_coef(y_true, y_pred):
    y_true_f = k.flatten(y_true)
    y_pred_f = k.flatten(y_pred)
    intersection = k.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (k.sum(y_true_f) + k.sum(y_pred_f) - intersection + 1.0)

def iou_coef(y_true, y_pred, smooth=1):
  intersection = k.sum(k.abs(y_true * y_pred), axis=[1,2])
  union = k.sum(y_true,[1,2])+k.sum(y_pred,[1,2])-intersection
  iou = k.mean((intersection+smooth) / (union+smooth), axis=0)
  return iou

def recall_m(y_true, y_pred):
    true_positives = k.sum(k.round(k.clip(y_true * y_pred, 0, 1)))
    possible_positives = k.sum(k.round(k.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + k.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = k.sum(k.round(k.clip(y_true * y_pred, 0, 1)))
    predicted_positives = k.sum(k.round(k.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + k.epsilon())
    return precision

def f1_m(y_true, y_pred):
   precision = precision_m(y_true, y_pred)
   recall = recall_m(y_true, y_pred)
   return 2*((precision*recall)/(precision+recall+k.epsilon()))

In [ ]:
def U_Net_Segmentation(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):

    inputs = Input(input_size)
    n = Lambda(lambda x:x/255)(inputs)


    c1 = Conv2D(32, (3,3), activation='relu', padding='same', kernel_initializer=tf.keras.initializers.Ones())(n)
    #c1 = BatchNormalization()(c1)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(32, (3,3), activation='relu', padding='same')(c1)
    #c1 = BatchNormalization()(c1)


    p1 = MaxPooling2D((2,2))(c1)


    c2 = Conv2D(64, (3,3), activation='relu', padding='same')(p1)
    #c2 = BatchNormalization()(c2)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(64, (3,3), activation='relu', padding='same')(c2)
    #c2 = BatchNormalization()(c2)


    p2 = MaxPooling2D((2,2))(c2)


    c3 = Conv2D(128, (3,3), activation='relu', padding='same')(p2)
    #c3 = BatchNormalization()(c3)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3,3), activation='relu', padding='same')(c3)
    #c3 = BatchNormalization()(c3)


    p3 = MaxPooling2D((2,2))(c3)


    c4 = Conv2D(256, (3,3), activation='relu', padding='same')(p3)
    #c4 = BatchNormalization()(c4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(256, (3,3), activation='relu', padding='same')(c4)
    #c4 = BatchNormalization()(c4)


    p4 = MaxPooling2D((2,2))(c4)


    c5 = Conv2D(512, (3,3), activation='elu', padding='same')(p4)
    #c5 = BatchNormalization()(c5)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(512, (3,3), activation='relu', padding='same')(c5)
    #c5 = BatchNormalization()(c5)


    u6 = Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c5)
    u6 = concatenate([u6, c4])


    c6 = Conv2D(256, (3,3), activation='relu', padding='same')(u6)
    #c6 = BatchNormalization()(c6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(256, (3,3), activation='relu', padding='same')(c6)
    #c6 = BatchNormalization()(c6)


    u7 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c6)
    u7 = concatenate([u7, c3])


    c7 = Conv2D(128, (3,3), activation='relu', padding='same')(u7)
    #c7 = BatchNormalization()(c7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(128, (3,3), activation='relu', padding='same')(c7)
    #c7 = BatchNormalization()(c7)


    u8 = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
    u8 = concatenate([u8, c2])


    c8 = Conv2D(64, (3,3), activation='relu', padding='same')(u8)
    #c8 = BatchNormalization()(c8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(64, (3,3), activation='relu', padding='same')(c8)
    #c8 = BatchNormalization()(c8)


    u9 = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis = 3)


    c9 = Conv2D(32, (3,3), activation='relu', padding='same')(u9)
    #c9 = BatchNormalization()(c9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(32, (3,3), activation='relu', padding='same')(c9)
    #c9 = BatchNormalization()(c9)

    outputs = Conv2D(1,(1,1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss = ['binary_crossentropy'], metrics=[tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0, 1]), tf.keras.metrics.IoU(num_classes=2, target_class_ids=[0]), tf.keras.metrics.IoU(num_classes=2, target_class_ids=[1]), f1_m ,precision_m , recall_m, 'accuracy', jacard_coef])
    model.summary()
    return model

In [ ]:
imshow(Train_Input[15])

In [ ]:
model = U_Net_Segmentation()

In [ ]:
# 7. Show The Results per Epoch

class loss_history(keras.callbacks.Callback):

    def __init__ (self, x=4):
        self.x = x

    def on_epoch_begin(self, epoch, logs={}):

        imshow(np.squeeze(Train_Input[self.x]))
        plt.show()

        imshow(np.squeeze(Train_Mask[self.x]), cmap='Greys_r')
        plt.show()

        preds_train = self.model.predict(np.expand_dims(Train_Input[self.x], axis = 0))
        imshow(np.squeeze(preds_train[0]), cmap='Greys_r')
        plt.show()

Model_Path = 'gdrive/My Drive/Saved_Models/2D_U_Net/10Parker_IoU_F1_JC_ACC_Modified_U_Net_BCE_001'

earlystopper = EarlyStopping(patience=7, verbose=1)
checkpointer = ModelCheckpoint(Model_Path, verbose = 1, save_best_only=True)

In [ ]:
Validation_Input = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Validation_Dataset/Image_512x512/*.png"))]
Validation_Mask = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Validation_Dataset/Label_512x512/*.png"))]

Validation_Input = np.array(Validation_Input)
Validation_Mask = np.array(Validation_Mask)

Validation_Mask = cv2.normalize(Validation_Mask, None, alpha=1,beta=0, norm_type=cv2.NORM_MINMAX)

In [ ]:
Validation_Input.shape

In [ ]:
# 8. Train U_NET Model using Training Samples

results = model.fit(Train_Input, Train_Mask,
                    validation_split = 0.2,
                    batch_size=10,
                    epochs=150
                    , callbacks=[earlystopper, checkpointer, loss_history()])

#results = model.fit(Train_Input, Train_Mask,
#                    validation_data=(Validation_Input, Validation_Mask),
#                   batch_size=10,
#                    epochs=150
#                    , callbacks=[earlystopper, checkpointer, loss_history()])

#earlystopper

In [ ]:
preds_train = model.predict(Train_Input, batch_size=10)
preds_train_t = preds_train.astype(np.uint8)
#preds_test = model.predict(Test_Inputs, verbose=1)
#preds_test_t = (preds_test>0.5).astype(np.uint8)

In [ ]:
# 10. Show Final Results (Segmented Images)

ix = random.randint(0, len(Train_Input)-1)

print(ix)

print('Train_Image')
imshow(Train_Input[ix])
plt.show()

print('Train_Mask')
imshow(np.squeeze(Train_Mask[ix]), cmap='Greys_r')
plt.show()

print('Segmented_Image')
imshow(np.squeeze(preds_train[ix]))
plt.show()

In [ ]:
# 11. Show Loss and ACC Plots


# 11.1. Summarize History for Loss

plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()


# 11.1. Summarize History for IOU

plt.plot(results.history['io_u'])
plt.plot(results.history['val_io_u'])
plt.title('iou_coef')
plt.ylabel('IOU')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for IOU

plt.plot(results.history['io_u_1'])
plt.plot(results.history['val_io_u_1'])
plt.title('iou_coef')
plt.ylabel('IOU')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for IOU

plt.plot(results.history['io_u_2'])
plt.plot(results.history['val_io_u_2'])
plt.title('iou_coef')
plt.ylabel('IOU')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for Precision

plt.plot(results.history['precision_m'])
plt.plot(results.history['val_precision_m'])
plt.title('Precision')
plt.ylabel('Prec')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for Recall

plt.plot(results.history['recall_m'])
plt.plot(results.history['val_recall_m'])
plt.title('Recal')
plt.ylabel('Recl')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for F1_Score

plt.plot(results.history['f1_m'])
plt.plot(results.history['val_f1_m'])
plt.title('F1 Score')
plt.ylabel('F1')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for Jaccard Coefficient

plt.plot(results.history['jacard_coef'])
plt.plot(results.history['val_jacard_coef'])
plt.title('Jaccard Coef')
plt.ylabel('JC')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

# 11.1. Summarize History for Accuracy

plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('Accuracy')
plt.ylabel('ACC')
plt.xlabel('epochs')
plt.legend(['Training','Validation'], loc = 'upper left')
plt.show()

In [ ]:
Test_Input = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Berea_Sand_Texas/Test_Image_Berea_512/*.png"))]
Test_Mask = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Berea_Sand_Texas/Test_Mask_Berea_512/*.png"))]

Test_Input = np.array(Test_Input)
Test_Mask = np.array(Test_Mask)

Test_Mask = cv2.normalize(Test_Mask, None, alpha=1,beta=0, norm_type=cv2.NORM_MINMAX)

In [ ]:
preds_test = model.predict(Test_Input, verbose=1)
preds_test_t = preds_test.astype(np.uint8)

In [ ]:
# 10. Show Final Results (Segmented Images)

ix = random.randint(0, len(Train_Input)-1)

print(ix)

print('Train_Image')
imshow(Train_Input[ix])
plt.show()

print('Train_Mask')
imshow(np.squeeze(Train_Mask[ix]), cmap='Greys_r')
plt.show()

print('Segmented_Image')
imshow(np.squeeze(preds_train[ix]))
plt.show()


iix = random.randint(0, len(Test_Input)-1)

print(ix)

print('Test_Image')
imshow(Test_Input[iix])
plt.show()

print('Test_Mask')
imshow(np.squeeze(Test_Mask[iix]), cmap='Greys_r')
plt.show()

print('Segmented_Image')
imshow(np.squeeze(preds_test[iix]))
plt.show()

In [ ]:
preds_test_t = (preds_test>0.5).astype(np.uint8)

In [ ]:
imshow(np.squeeze(preds_test_t[215]), cmap='Greys_r')
plt.show()

In [ ]:
imshow(np.squeeze(Test_Mask[215]), cmap='Greys_r')
plt.show()

In [ ]:
Diff_Image = cv2.subtract(preds_test_t[215], Test_Mask[215])

In [ ]:
print('Difference Image')
imshow(np.squeeze(Diff_Image), cmap='Greys')
plt.show()

In [ ]:
intersection = np.logical_and(Test_Mask[215], preds_test_t[215])
union = np.logical_or(Test_Mask[215], preds_test_t[215])
iou_score = np.sum(intersection) / np.sum(union)

In [ ]:
iou_score

In [ ]:
Test_Input = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Buff_Berea_Sand/Image_512x512/*.png"))]
Test_Mask = [cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in sorted(glob.glob("gdrive/My Drive/Buff_Berea_Sand/Label_512x512/*.png"))]

Test_Input = np.array(Test_Input)
Test_Mask = np.array(Test_Mask)

Test_Mask = cv2.normalize(Test_Mask, None, alpha=1,beta=0, norm_type=cv2.NORM_MINMAX)

In [ ]:
preds_test = model.predict(Test_Input, verbose=1)
preds_test_t = (preds_test>0.5).astype(np.uint8)

In [ ]:
iix = random.randint(0, len(Test_Input)-1)

print(ix)

print('Test_Image')
imshow(Test_Input[iix])
plt.show()

print('Test_Mask')
imshow(np.squeeze(Test_Mask[iix]), cmap='Greys_r')
plt.show()

print('Segmented_Image')
imshow(np.squeeze(preds_test[iix]))
plt.show()

In [ ]:
Diff_Image = cv2.subtract(Test_Mask[215], preds_test_t[215])

In [ ]:
print('Difference Image')
imshow(np.squeeze(Diff_Image), cmap='Greys')
plt.show()